# Building VRS objects for Phenopackets v2

This notebook generates VRS objects for use in Phenopackets, as described in *GA4GH Phenopackets: A practical introduction*.

First, we will want to import VRS Python, the reference implementation for generating VRS objects. We are going to use the `0.7` version of VRS Python, which corresponds to VRS version `1.2`, as used in Phenopackets.

We are also going to use the VRS python Extras, including the SeqRepo package for sequence lookups.

In [1]:
from ga4gh import vrs, core
from ga4gh.vrs.dataproxy import SeqRepoDataProxy
from biocommons.seqrepo import SeqRepo
from ga4gh.vrs.extras.translator import Translator
from copy import deepcopy

dp = SeqRepoDataProxy(SeqRepo('/usr/local/share/seqrepo/latest'))
tlr = Translator(dp)

Removing allOf attribute from CopyNumber to avoid python-jsonschema-objects error.
Removing allOf attribute from SequenceInterval to avoid python-jsonschema-objects error.
Removing allOf attribute from RepeatedSequenceExpression to avoid python-jsonschema-objects error.
/Users/ahw001/.local/share/virtualenvs/vrs-phenopackets-4Z1ga_iF/lib/python3.9/site-packages/python_jsonschema_objects/__init__.py:49: UserWarning: Schema version http://json-schema.org/draft-07/schema not recognized. Some keywords and features may not be supported.
  warnings.warn(


In [2]:
# NOTE: VRS here is the VRS Python package, not the VRS specification; 
# accordingly vrs.__version__ is the VRS Python version. 
# Phenopackets v2 requires VRS Python v0.7.x, which is based on VRS 1.2
vrs.__version__

'0.7.4'

## Example 1
> ... The detected values were relatively low but within the normal range and a complete RB1 deletion in mosaicism was suspected. A genomic SNP array (AffymetrixCytoScan 750 array) was performed and a 13q deletion of 35.7 Mb from 13q12.13 to 13q21.2 (arr[hg19] 13q12.13q21.2(26,555,387–62,280,955) × 1–2) detected in around 40% of cells was confirmed.

This example will demonstrate how to specify the above using VRS and VRSATILE.

First, we identify the sequence on which the deletion occurred. We could start by retrieving the sequence identifier for [GRCh37 chromosome 13](https://www.ncbi.nlm.nih.gov/nuccore/NC_000013.10):

In [3]:
def translate_sequence_alias_to_ga4gh_id(sequence_alias):
    translated_ids = dp.translate_sequence_identifier(sequence_alias,'ga4gh')
    assert len(translated_ids) == 1 # Any given sequence should only have 1 GA4GH-namespaced identifier
    return translated_ids[0]
    
chr13_grch37_ga4gh_seq_id = translate_sequence_alias_to_ga4gh_id('GRCh37:chr13')
chr13_grch37_ga4gh_seq_id

'ga4gh:SQ.Ewb9qlgTqN6e_XQiRVYpoUfZJHXeiUfH'

The above sequence ID is useful to capture in VRS messages because it is data-driven; it can be derived from your local reference sequence and guarantees equivalence across all identical sequences regardless of accession. We can see a few of these with a metadata lookup:

In [4]:
dp.get_metadata('ga4gh:SQ.Ewb9qlgTqN6e_XQiRVYpoUfZJHXeiUfH')

{'length': 115169878,
 'alphabet': 'ACGNT',
 'added': '2016-08-24T05:10:23Z',
 'aliases': ['GRCh37:13',
  'GRCh37:chr13',
  'GRCh37.p10:13',
  'GRCh37.p10:chr13',
  'GRCh37.p11:13',
  'GRCh37.p11:chr13',
  'GRCh37.p12:13',
  'GRCh37.p12:chr13',
  'GRCh37.p13:13',
  'GRCh37.p13:chr13',
  'GRCh37.p2:13',
  'GRCh37.p2:chr13',
  'GRCh37.p5:13',
  'GRCh37.p5:chr13',
  'GRCh37.p9:13',
  'GRCh37.p9:chr13',
  'MD5:283f8d7892baa81b510a015719ca7b0b',
  'NCBI:NC_000013.10',
  'refseq:NC_000013.10',
  'SEGUID:oYfOuGjgq2YenMDoPeWgsO+hJRI',
  'SHA1:a187ceb868e0ab661e9cc0e83de5a0b0efa12512',
  'VMC:GS_Ewb9qlgTqN6e_XQiRVYpoUfZJHXeiUfH',
  'sha512t24u:Ewb9qlgTqN6e_XQiRVYpoUfZJHXeiUfH',
  'ga4gh:SQ.Ewb9qlgTqN6e_XQiRVYpoUfZJHXeiUfH',
  'hs37-1kg:13',
  'hs37d5:13']}

With our knowledge of the sequence identifier, we can now define an interval on that sequence to describe the location of the deletion:

In [5]:
location_start = vrs.models.Number(value=26555377) # NOTE: VRS uses inter-residue coordinates
location_end   = vrs.models.Number(value=62280955)
interval = vrs.models.SequenceInterval(start=location_start, end=location_end)

deletion_location = vrs.models.SequenceLocation(sequence_id=chr13_grch37_ga4gh_seq_id, interval=interval)

From there, we describe a single-copy state of the location using a CopyNumber variation, along with its globally-unique computed identifier:

In [6]:
dsl = vrs.models.DerivedSequenceExpression(location=deletion_location, reverse_complement=False)
cnv = vrs.models.CopyNumber(subject=dsl, copies=vrs.models.Number(value=1))
cnv._id = core.ga4gh_identify(cnv)
cnv.as_dict()

{'_id': 'ga4gh:VCN.AFfJws1M4Lg8w1O3XknmHYc9TU2hHYpp',
 'type': 'CopyNumber',
 'subject': {'type': 'DerivedSequenceExpression',
  'location': {'type': 'SequenceLocation',
   'sequence_id': 'ga4gh:SQ.Ewb9qlgTqN6e_XQiRVYpoUfZJHXeiUfH',
   'interval': {'type': 'SequenceInterval',
    'start': {'type': 'Number', 'value': 26555377},
    'end': {'type': 'Number', 'value': 62280955}}},
  'reverse_complement': False},
 'copies': {'type': 'Number', 'value': 1}}

It is also allowed, but discouraged, to have VRS messages that use accessioned identifiers (e.g. NCBI:NC_000013.10) or even Genome Reference Consortium chromosome references for the `sequence_id` attribute, like so:

In [7]:
deletion_labeled_location = vrs.models.SequenceLocation(sequence_id='GRCh37:chr13', interval=interval)
dsl_labeled = vrs.models.DerivedSequenceExpression(location=deletion_labeled_location, reverse_complement=False)
cnv_labeled = vrs.models.CopyNumber(subject=dsl_labeled, copies=vrs.models.Number(value=1))
cnv_labeled.as_dict()

{'type': 'CopyNumber',
 'subject': {'type': 'DerivedSequenceExpression',
  'location': {'type': 'SequenceLocation',
   'sequence_id': 'GRCh37:chr13',
   'interval': {'type': 'SequenceInterval',
    'start': {'type': 'Number', 'value': 26555377},
    'end': {'type': 'Number', 'value': 62280955}}},
  'reverse_complement': False},
 'copies': {'type': 'Number', 'value': 1}}

An alternative approach to this is to use Variation Descriptors, part of the [VRSATILE Value Object Descriptor specification](https://vrsatile.readthedocs.io/en/latest/value_object_descriptor/index.html). Phenopackets has a [definition of Variation Descriptors as implemented in protobuf](https://github.com/phenopackets/phenopacket-schema/blob/master/src/main/proto/ga4gh/vrsatile/v1/vrsatile.proto). We will create a JSON message that leverages both the computable CNV representation from VRS, as well as relevant human-readable expressions and annotations for the event with VRSATILE:

In [8]:
vrsatile_cnv = {
    'id': 'cnv001',
    'variation': cnv.as_dict(),
    'label': 'arr[hg19] 13q12.13q21.2(26,555,387–62,280,955) × 1–2',
    'molecule_context': 'genomic',
    'expressions': [{
        'syntax': 'iscn',
        'value': 'del(13)(p12.13p21.2)'
    },{
        'syntax': 'hgvs.g',
        'value': 'NC_000013.10:g.(26555387_62280955)del',
        'version': '20.05'
    }],
    'extensions': [{
        'name': 'chromosome',
        'value': '13'
    },{
        'name': 'cellularity',
        'value': 0.40
    },{
        'name': 'genome_reference',
        'value': 'GRCh37'
    }]
}
vrsatile_cnv

{'id': 'cnv001',
 'variation': {'_id': 'ga4gh:VCN.AFfJws1M4Lg8w1O3XknmHYc9TU2hHYpp',
  'type': 'CopyNumber',
  'subject': {'type': 'DerivedSequenceExpression',
   'location': {'type': 'SequenceLocation',
    'sequence_id': 'ga4gh:SQ.Ewb9qlgTqN6e_XQiRVYpoUfZJHXeiUfH',
    'interval': {'type': 'SequenceInterval',
     'start': {'type': 'Number', 'value': 26555377},
     'end': {'type': 'Number', 'value': 62280955}}},
   'reverse_complement': False},
  'copies': {'type': 'Number', 'value': 1}},
 'label': 'arr[hg19] 13q12.13q21.2(26,555,387–62,280,955) × 1–2',
 'molecule_context': 'genomic',
 'expressions': [{'syntax': 'iscn', 'value': 'del(13)(p12.13p21.2)'},
  {'syntax': 'hgvs.g',
   'value': 'NC_000013.10:g.(26555387_62280955)del',
   'version': '20.05'}],
 'extensions': [{'name': 'chromosome', 'value': '13'},
  {'name': 'cellularity', 'value': 0.4},
  {'name': 'genome_reference', 'value': 'GRCh37'}]}

**NOTE:** Location-based CopyNumberVariation subjects are currently described using DerivedSequenceExpression as shown above, but we will also be allowing Location as a direct subject in an upcoming minor version release of VRS; we believe this is more intuitive for users. We will update this notebook if / when those changes are integrated into Phenopackets.

## Example 2
>Looking for second hit mutations in RB1, we applied a custom designed NGS panel (Onconano V2) that included the RB1, BCOR and CREBPP genes (among other 400 commonly mutated genes in pediatric cancer). The study detected only one pathogenic single-nucleotide variant, RB1 c.958C>T (p.Arg320Ter) (NM_000321.2 chromosomal position 13–48,941,648-C-T; allele frequency of 25%).

For this example, we model the state of the sequence (an observed thymine) at a chromosomal location as an `Allele`:

In [9]:
location_start = vrs.models.Number(value=48941647)
location_end = vrs.models.Number(value=48941648)
substitution_interval = vrs.models.SequenceInterval(start=location_start, end=location_end)

transcript_sequence_id = translate_sequence_alias_to_ga4gh_id('refseq:NM_000321.2')

location = vrs.models.SequenceLocation(sequence_id=transcript_sequence_id, interval=substitution_interval)
state = vrs.models.LiteralSequenceExpression(sequence='T')

substitution = vrs.models.Allele(location=location, state=state)
substitution._id = core.ga4gh_identify(substitution)

substitution.for_json()

{'_id': 'ga4gh:VA.GuPzvZoansqNHPoXkQLXKo31VkTpDKsM',
 'type': 'Allele',
 'location': {'type': 'SequenceLocation',
  'sequence_id': 'ga4gh:SQ.FOWokFmA__GgqWLtqFoWWDLuNEvvGwIJ',
  'interval': {'type': 'SequenceInterval',
   'start': {'type': 'Number', 'value': 48941647},
   'end': {'type': 'Number', 'value': 48941648}}},
 'state': {'type': 'LiteralSequenceExpression', 'sequence': 'T'}}

Again, we can extend this with VRSATILE for relevant annotations:

In [10]:
vrsatile_substitution = {
    'id': 'allele001',
    'variation': substitution.as_dict(),
    'label': 'RB1 c.958C>T (p.Arg320Ter)',
    'molecule_context': 'transcript',
    'gene_context': {
        'value_id': 'hgnc:9884',
        'symbol': 'RB1',
    },
    'expressions': [{
        'syntax': 'hgvs.c',
        'value': 'NM_000321.2:c.958C>T',
        'version': '20.05'
    }],
    'extensions': [{
        'name': 'Allele Frequency',
        'value': 0.25
    },{
        'name': 'transcript_reference',
        'value': 'NM_000321.2'
    }]
}
vrsatile_substitution

{'id': 'allele001',
 'variation': {'_id': 'ga4gh:VA.GuPzvZoansqNHPoXkQLXKo31VkTpDKsM',
  'type': 'Allele',
  'location': {'type': 'SequenceLocation',
   'sequence_id': 'ga4gh:SQ.FOWokFmA__GgqWLtqFoWWDLuNEvvGwIJ',
   'interval': {'type': 'SequenceInterval',
    'start': {'type': 'Number', 'value': 48941647},
    'end': {'type': 'Number', 'value': 48941648}}},
  'state': {'type': 'LiteralSequenceExpression', 'sequence': 'T'}},
 'label': 'RB1 c.958C>T (p.Arg320Ter)',
 'molecule_context': 'transcript',
 'gene_context': {'value_id': 'hgnc:9884', 'symbol': 'RB1'},
 'expressions': [{'syntax': 'hgvs.c',
   'value': 'NM_000321.2:c.958C>T',
   'version': '20.05'}],
 'extensions': [{'name': 'Allele Frequency', 'value': 0.25},
  {'name': 'transcript_reference', 'value': 'NM_000321.2'}]}